<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Word-Count" data-toc-modified-id="Word-Count-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Word Count</a></span><ul class="toc-item"><li><span><a href="#Using-Counter" data-toc-modified-id="Using-Counter-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Using <code>Counter</code></a></span></li><li><span><a href="#Adding-Word-Counts-From-Two-Distinct-Datasets-Together" data-toc-modified-id="Adding-Word-Counts-From-Two-Distinct-Datasets-Together-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Adding Word Counts From Two Distinct Datasets Together</a></span></li></ul></li><li><span><a href="#Removing-Stopwords-Using-gensim" data-toc-modified-id="Removing-Stopwords-Using-gensim-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Removing Stopwords Using <code>gensim</code></a></span></li><li><span><a href="#Finding-Similar-Word-Matches-Using-difflib" data-toc-modified-id="Finding-Similar-Word-Matches-Using-difflib-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Finding Similar Word Matches Using <code>difflib</code></a></span><ul class="toc-item"><li><span><a href="#Fuzzy-Matching" data-toc-modified-id="Fuzzy-Matching-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Fuzzy Matching</a></span><ul class="toc-item"><li><span><a href="#Use-Cases" data-toc-modified-id="Use-Cases-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Use Cases</a></span></li><li><span><a href="#Limitations" data-toc-modified-id="Limitations-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Limitations</a></span></li><li><span><a href="#Slow-Performance" data-toc-modified-id="Slow-Performance-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Slow Performance</a></span></li><li><span><a href="#Not-&quot;Language-Aware&quot;" data-toc-modified-id="Not-&quot;Language-Aware&quot;-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Not "Language Aware"</a></span></li></ul></li><li><span><a href="#Install-the-Dependencies-if-Necessary" data-toc-modified-id="Install-the-Dependencies-if-Necessary-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Install the Dependencies if Necessary</a></span><ul class="toc-item"><li><span><a href="#Token-Set-Ratio-(following-examples-from-fuzzywuzzy's-documentation)" data-toc-modified-id="Token-Set-Ratio-(following-examples-from-fuzzywuzzy's-documentation)-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Token Set Ratio (following examples from <code>fuzzywuzzy</code>'s documentation)</a></span></li><li><span><a href="#Token-Set-Ratio" data-toc-modified-id="Token-Set-Ratio-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Token Set Ratio</a></span></li></ul></li></ul></li></ul></div>

# Word Count
## Using `Counter`

A normal dictionary object will return a key error if you do not first initialize the key value:

In [4]:
# error because you need to initialize
from typing import Dict
ordinary_dict: Dict = dict()
ordinary_dict["yu"] += 1

KeyError: 'yu'

The `Counter` object in `collections` has a default value of 0 for every key.

In [6]:
from collections import Counter

counter = Counter()
counter["yu"] += 1
counter

Counter({'yu': 1})

Moreover, the you can pass in a list of strings to the `Counter` constructor, as well as calling the
`most_common` method to get the most common words:

In [20]:
from typing import List
words: List[str] = open("tale-of-two-cities.txt").read().split()
dickens_counter = Counter(words)
dickens_counter.most_common(5)

[('the', 7363), ('and', 4727), ('of', 3944), ('to', 3398), ('a', 2792)]

You can also quickly use this counter to find the percentage of words in a corpus that belong to a certain word:

In [21]:
dickens_counter["the"] / sum(dickens_counter.values())

0.054036400998091885

## Adding Word Counts From Two Distinct Datasets Together

We can add two `Counter` objects together to get their combined counts. In this example, we'll load in the `fraudulent_emails.txt` dataset and start a new counter called `email_counter`.

In [23]:
email_counter = Counter(open("fraudulent_emails.txt").read().split())
email_counter.most_common(5)

[('the', 141), ('to', 116), ('I', 115), ('of', 80), ('in', 80)]

In [25]:
combined_counter: Counter = dickens_counter + email_counter
combined_counter.most_common(5)

[('the', 7504), ('and', 4789), ('of', 4024), ('to', 3514), ('a', 2834)]

You can also subtract counts from one dataset:

In [29]:
# get back the original email_counter
(combined_counter - dickens_counter).most_common(5)

[('the', 141), ('to', 116), ('I', 115), ('of', 80), ('in', 80)]

# Removing Stopwords Using `gensim`

Removing stopwords in `nltk` often means you first have to tokenize the document into distinct tokens, then run each token through to check if it is a stopword. Another commonly used NLP library in Python, `gensim`, has a helper function to do this all in one go:

In [2]:
from gensim.parsing.preprocessing import remove_stopwords

text = '''
Rendered in a manner desperate, by her state and by the beckoning of their conductor,
he drew over his neck the arm that shook upon his shoulder, lifted her a little, and hurried 
her into the room. He sat her down just within the door, and held her, clinging to him.
'''
processed_text = remove_stopwords(text)
processed_text

'Rendered manner desperate, state beckoning conductor, drew neck arm shook shoulder, lifted little, hurried room. He sat door, held her, clinging him.'

Note, however, this only works well if you are happy with Gensim's only predefined list of stopwords. To inspect what stopwords are used in Gensim, use
```python
from gensim.parsing.preprocessing import STOPWORDS
print(STOPWORDS)
```

# Finding Similar Word Matches Using `difflib`

Within Python's Standard Library, the `difflib` has a variety of tools for helping identify differences between text and content. It uses an algorithm called the **Ratcliff-Obershelp algorithm**, which is described in brief below:

> The idea is to find the longest contiguous matching subsequence that contains no “junk” elements; these “junk” elements are ones that are uninteresting in some sense, such as blank lines or whitespace. (Handling junk is an extension to the Ratcliff and Obershelp algorithm.) The same idea is then applied recursively to the pieces of the sequences to the left and to the right of the matching subsequence. This does not yield minimal edit sequences, but does tend to yield matches that “look right” to people. [Link](https://docs.python.org/3/library/difflib.html)

In [3]:
# this loads in the top 20k most popular words in the English language
words = set(map(lambda word: word.replace("\n", ""), open("20k.txt").readlines()))

In [4]:
# fuzzy matching / edit distance / common misspellings
import difflib

w = "knaght"
difflib.get_close_matches(w, words)

['knight', 'naughty', 'knights']

You can combine this with a tokenizer to create your own (very basic) spellcheck function:

In [5]:
from nltk.tokenize import word_tokenize

def spellcheck_document(text):
    new_tokens = []
    for token in word_tokenize(text):
        matches = difflib.get_close_matches(token.lower(), words, n=1, cutoff=0.7)
        if len(matches) == 0 or token.lower() in words:
            new_tokens.append(token)
        else:
            new_tokens.append(matches[0])
    return " ".join(new_tokens)
spellcheck_document("He is a craezy perzon")

'He is a crazy person'

## Fuzzy Matching

Fuzzy matching refers to "approximate matching", where we are allowed a certain degree of error between the query value and the search result. 

The `fuzzywuzzy` library uses a distance measure called **Levenshtein Distance** which describes the minimum number of operations to transform one string into another.

* `cat` $\rightarrow$ `cat` : `0` distance
* `dog` $\rightarrow$ `door`: `2` distance

### Use Cases

* spell checking
* DNA analysis
* authorship/plagiarism detection

### Limitations

### Slow Performance

In [1]:
# this loads in the top 20k most popular words in the English language
words = set(map(lambda word: word.replace("\n", ""), open("20k.txt").readlines()))

In [42]:
from timeit import default_timer as timer
from fuzzywuzzy import process

target = "kerfuffled"


start = timer()
for i in range(10):
    bests = process.extractBests(target, words, scorer=fuzz.ratio)
end = timer()
print(end - start) # Time in seconds to check 10 words
print(f'Best results: {bests}')

0.8788837209999656
Best results: [('ruled', 67), ('fulfilled', 63), ('stuffed', 59), ('refused', 59), ('perfume', 59)]


In [43]:
fuzz.ratio("kerfuffled", "ruled")

67

### Not "Language Aware"

> Comparing the classification proposed by the Levenshtein distance to that of the comparative method shows that the Levenshtein classification is correct only 40% of time. Standardizing the orthography increases the performance, but only to a maximum of 65% accuracy within language subgroups. The accuracy of the Levenshtein classification **decreases rapidly with phylogenetic distance**, failing to discriminate homology and chance similarity across distantly related languages.This poor performance suggests the need for more linguistically nuanced methods for automated language classification tasks.

["Levenshtein distances fail to identify language relationships accurately" by Simon Greenhill](https://dl.acm.org/doi/10.1162/COLI_a_00073)

## Install the Dependencies if Necessary
```python
!pip3 install fuzzywuzzy
!pip3 install python-Levenshtein
```

In [2]:
!pip3 install fuzzywuzzy

In [4]:
!pip install python-Levenshtein

  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\19258\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\19258\\AppData\\Local\\Temp\\pip-install-1ujgcgkt\\python-levenshtein_c52381c406004e08bc98b99189729c1a\\setup.py'"'"'; __file__='"'"'C:\\Users\\19258\\AppData\\Local\\Temp\\pip-install-1ujgcgkt\\python-levenshtein_c52381c406004e08bc98b99189729c1a\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\19258\AppData\Local\Temp\pip-wheel-acvqy8zl'
       cwd: C:\Users\19258\AppData\Local\Temp\pip-install-1ujgcgkt\python-levenshtein_c52381c406004e08bc98b99189729c1a\
  Complete output (27 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\Levenshtein


    Running setup.py install for python-Levenshtein: started
    Running setup.py install for python-Levenshtein: finished with status 'error'


  copying Levenshtein\StringMatcher.py -> build\lib.win-amd64-3.7\Levenshtein
  copying Levenshtein\__init__.py -> build\lib.win-amd64-3.7\Levenshtein
  running egg_info
  writing python_Levenshtein.egg-info\PKG-INFO
  writing dependency_links to python_Levenshtein.egg-info\dependency_links.txt
  writing entry points to python_Levenshtein.egg-info\entry_points.txt
  writing namespace_packages to python_Levenshtein.egg-info\namespace_packages.txt
  writing requirements to python_Levenshtein.egg-info\requires.txt
  writing top-level names to python_Levenshtein.egg-info\top_level.txt
  reading manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  writing manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  copying Levenshtein\_levenshtein.c -> build\lib.win-amd64-3.7\Levenshtein
  copying Levenshtein\_levenshtein.h -> build\lib.win-amd64-3.7\Levenshtein
  running build_ext
  building 'Levenshtein._levenshtein' extension
  error: Microsoft

In [6]:
from fuzzywuzzy import fuzz

In [7]:
fuzz.ratio("cat", "saturday")

36

In [8]:
fuzz.ratio("dog", "cat")

0

In [9]:
fuzz.ratio("dog", "hog")

67

In [20]:
fuzz.ratio("smithy", "smithfield")

62

In [23]:
# is it symmetric?
fuzz.ratio("smithfield", "smithy")

62

In [24]:
fuzz.ratio("photosynthesis", "photosynthetic")

86

In [25]:
# does case matter?
fuzz.ratio("Photosynthesis", "photosynthetic")

79

In [26]:
# what happens if you arbitrarily increase the size of the strings?

fuzz.ratio("dog" * 3, "hog" * 3)

67

### Token Set Ratio (following examples from `fuzzywuzzy`'s documentation)

In [44]:
print(fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear"))
print(fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear"))
# set = unqiue

84
100


### Token Set Ratio

In [46]:
print(fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))
print(fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))

91
100
